## Estructura de los Datos

### Cash_Request (CR)

##### CR.Status (23970 registros)

- **money_back**: 16397 registros. El CR fue reembolsado exitosamente.(The CR was successfully reimbursed.)
---
- **active**: 59 registros. Los fondos fueron recibidos en la cuenta del cliente. (Funds were received on the customer account.)
- **direct_debit_sent**: 34 registros. Se envió un débito directo SEPA, pero aún no se confirma el resultado. (We sent/scheduled a SEPA direct debit to charge the customer account. The result of this debit is not yet confirmed)
---
- **rejected**: 6568 registros. El CR necesitó una revisión manual y fue rechazado. (The CR needed a manual review and was rejected)
- **direct_debit_rejected**: 831 registros. El intento de débito directo SEPA falló. (Our last attempt of SEPA direct debit to charge the customer was rejected )
- **transaction_declined**: 48 registros. No se pudo enviar el dinero al cliente. (We failed to send the funds to the customer) 
- **canceled**: 33 registros. El usuario no confirmó el CR en la app, fue cancelado automáticamente. (The user didn't confirm the cash request in-app, we automatically canceled it)

---
**En los datos proporcionados, NO aparecen los valores:** 
- approved : CR is a 'regular' one (= without fees) and was approved either automatically or manually. Funds will be sent aprox. 7 days after the creation
- money_sent : We transferred the fund to the customer account. Will change to active once we detect that the user received the funds (using user's bank history)
- pending : The CR is pending a manual review from an analyst
- waiting_user_confirmation : The user needs to confirm in-app that he want the CR (for legal reasons)
- waiting_reimbursement : We were not able to estimate a date of reimbursement, the user needs to choose one in the app.


##### CR.Transfer Type

- **instant**: El usuario eligió recibir el adelanto instantáneamente. (user choose not received the advance instantly)
- **regular**: El usuario eligió no pagar inmediatamente y esperar la transferencia. (user choose to not pay and wait for the transfer)


##### CR.Recovery Status

- **null**: El CR nunca tuvo un incidente de pago.
- **completed**: El incidente de pago fue resuelto (el CR fue reembolsado).
---
- **pending**: El incidente de pago aún está abierto.
- **pending_direct_debit**: El incidente de pago sigue abierto, pero se ha lanzado un débito directo SEPA.


### Fees (FE)

##### FE.Type

- **instant_payment**: Fees por adelanto instantáneo. (fees for instant cash request (send directly after user's request, through SEPA Instant Payment) )
- **split_payment**: Fees por pago fraccionado (en caso de un incidente). (futures fees for split payment (in case of an incident, we'll soon offer the possibility to our users to reimburse in multiples installements))
- **incident**: Fees por fallos de reembolsos. (fees for failed reimbursement. Created after a failed direct debit)
- **postpone**: Fees por la solicitud de posponer un reembolso. (fees created when a user want to postpone the reimbursment of a CR)

##### FE.Status (= does the fees was successfully charged)

- **accepted**: El fee fue cobrado exitosamente. (fees were successfully charged)
- **confirmed**: El usuario completó una acción que creó un fee. (the user made an action who created a fee. It will normally get charged at the moment of the CR's reimbursement. In some rare cases, postpones are confirmed without being charges due to a commercial offer.)
---
- **rejected**: El último intento de cobrar el fee falló. (the last attempt to charge the fee failed.)
- **cancelled**: El fee fue creado pero cancelado por algún motivo. (fee was created and cancelled for some reasons. It's used to fix issues with fees but it mainly concern postpone fees who failed. We are charging the fees at the moment of the postpone request. If it failed, the postpone is not accepted and the reimbursement date still the same.)

##### FE.Category

- **rejected_direct_debit**: Fees creados cuando el banco del usuario rechaza el primer débito directo. (fees created when user's bank rejects the first direct debit)
- **month_delay_on_payment**: Fees creados cada mes hasta que el incidente se cierre. (fees created every month until the incident is closed)
---
- **null**: No figura a la documentacio

##### FE.paid_at:	

- Timestamp of the fee's payment

##### FE.charge_moment (When the fee will be charge).

- **before**: El fee se cobra en el momento de su creación. (the fee should be charged at the moment of its creation)
- **after**: El fee se cobra cuando el CR es reembolsado. (the fee should be charged at the moment of the CR's reimbursement)

##### FE.total_amount

- Amount of the fee (including VAT)

## Casos

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
#pm.help()
#pm.reset()
#pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df_jo = pm.df('df_jo')
#df_jo.info()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()

In [ ]:
display(df_jo)

In [ ]:
pd.options.display.max_columns = None
df = pm.df('df_jo')

# send_at mirar si tiene registros faltantes.
# money_back_date

#0 registros
#display(df[ (df['reimbursement_date'].isna()) & (df['stat_cr'] == 'money_back') ])


#191 registros
#display(df[ ((df['reimbursement_date'].isna()) | (df['money_back_date'].isna())) & (df['stat_cr'] == 'money_back') ])

#191 Normalizamos:
display(df[ (df['money_back_date'].isna()) & (df['stat_cr'] == 'money_back') ])
df['money_back_date'] = df.apply(
            lambda row: row['reimbursement_date']             
            if ( pd.isna(row['money_back_date']) & (row['stat_cr'] == 'money_back') ) 
            else row['money_back_date'], axis=1)
display(df[ (df['money_back_date'].isna()) & (df['stat_cr'] == 'money_back') ])

# 838  registros
#display(df[ (df['reimbursement_date'].notna()) & (df['money_back_date'].notna() & (df['stat_cr'] != 'money_back') )])#.head(5).reset_index()


In [ ]:
good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted']
df_jo['good'] = (df_jo['stat_cr'].isin(good_cr)) & (df_jo['stat_fe'].isin(good_fe))


In [ ]:
display(df_jo[df_jo.user_id ==2002].reset_index().head(10))

In [ ]:
pd.options.display.max_rows = None
df = pm.df('df_jo')
#df.info()

df_cr = pm.df('cr_cp')
df_cr.info()

grouped_counts = df.groupby(
    ['id_cr']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))


grouped_counts = df.groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'transfer_type', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))

grouped_counts = df.query('stat_fe == "accepted" & stat_cr == "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
#display(grouped_counts.head(10))


grouped_counts = df.query('stat_fe == "accepted" & stat_cr != "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
#display(grouped_counts.head(10))


In [ ]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)


In [ ]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)

In [ ]:
display(cohort_analysis_2.head(5))

In [ ]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            # #total_paid_fees=('fee', 'sum'),            
            # #Num_Solicitudes=('id_cr', 'nunique')
            # #Num_Solicitudes=('id_cr', lambda x: x.unique().sum())
            # Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())

            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

In [ ]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)

# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

In [ ]:
pd.options.display.max_rows = None
tops = df_jo[df_jo['stat_cr' ]== 'money_back'].groupby('user_id').agg(fees=('fee','sum'))
tops = df_jo[df_jo['stat_fe'] == 'accepted'  ].groupby('user_id').agg(fees=('fee','sum'))
#display(df_jo[tops])
top_users = tops.sort_values(by='fees', ascending=False).iloc[:10].reset_index()
display(top_users)
#top_users = tops.sort_values(by='fees', ascending=True).iloc[:10]
#display(top_users)

#display(df_jo[top_users])

In [ ]:
df = pm.df('df_jo')
timeFormat ='S' #D
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))
df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)
df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs','good_user',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]

user_id = 2002# 16391 # 2002, 1987, 13851, 16391, 102105
display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])

#print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-8177]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].sort_values(['created_at','created_at_fe']).reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[fields])

user_ids = [user_id] 
pd.options.display.max_columns = None
#print("Casos segun Cash User ID")
for id in (user_ids):
    df_t = df[(df['user_id'] == id)]#.reset_index()
    df_t = df_t[df_t['stat_cr'] == 'money_back']
    df_t = df_t[df_t['stat_fe'] == 'accepted']
    
    df_t = df_t.sort_values(['created_at','created_at_fe']).reset_index(drop=True)
    #df_t.set_index('id_cr', inplace=True)
    print(f"Only money_back - user_id {id}")
    display(df_t[fields])
    df_t = df[(df['user_id'] == id) ].sort_values(['created_at','created_at_fe']).reset_index(drop=True)

    print(f"user_id {id}")
    display(df_t[fields])

#user_ids = [13851] [2002] , 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.


In [ ]:
cohort_analysis_2 = (
    df_jo.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        # Sumar los valores de 'fee' donde 'stat_fe' es igual a 'accepted'
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        
        # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),

        # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
    )
)

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002]) ## 13851]) #.tail(60))

In [ ]:
# Filtrar només les files necessàries per les operacions
#filtered_df = df_jo[df_jo['stat_fe'] == 'accepted']
filtered_df = df_jo.copy()

# Agregar les dades agrupades
cohort_analysis_2 = (
    filtered_df.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        #total_paid_fees=('fee', 'sum'),
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique()),
        transfer_type=('transfer_type', lambda x: ', '.join(x.unique()))  # Llista valors únics
    )
)

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

In [ ]:
#cohort_analysis_3 = filtered_df.groupby(['user_id', 'Mes_created_at'])
#display(cohort_analysis_3)

In [ ]:
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr']
) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

# Resultado final del DataFrame de análisis de cohortes
#cohort_analysis_2
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

In [ ]:
#cohort_analysis_2.info
#cohort_analysis_2.count
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))


In [ ]:
# Crear columna 'Cohorte' que indica el mes de la primera solicitud de cada cliente
df_jo['Cohorte'] = df_jo.groupby('user_id')['created_at'].transform('min').dt.to_period('M')
#df_jo.head(10)

In [ ]:
# Agrupar los datos por cohorte y mes para calcular las métricas mensuales
cohort_analysis = df_jo.groupby(['Cohorte', 'Mes_created_at']).agg(
    Num_Clientes=('user_id', 'nunique'),  # Número de clientes únicos por cohorte y mes
    Num_Solicitudes=('id_cr', 'nunique'),  # Total de solicitudes únicas por cohorte y mes
    Solicitudes_Instant=('transfer_type', lambda x: (x == 'instant').sum()),  # Solicitudes tipo 'instant' por cohorte y mes
    Solicitudes_Regular=('transfer_type', lambda x: (x == 'regular').sum()),  # Solicitudes tipo 'regular' por cohorte y mes
    Clientes_con_fees=('fee', lambda x: (x > 0).sum()),  # Clientes con fees por cohorte y mes
    CR=('id_cr', lambda ids: df_jo[df_jo['id_cr'].isin(ids) & df_jo['cash_request_received_date'].notnull()]['amount'].sum()),

    Fees=('fee', 'sum'),  # Total de fees por cohorte y mes
    # Contar las líneas con 'stat_fe' == 'accepted'
    paid_fees=('stat_fe', lambda x: (x == 'accepted').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_fees=('stat_fe', lambda x: (x != 'accepted').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] != 'accepted'].sum()),
     # Contar las líneas con 'stat_cr' == 'money_back'
    paid_cr=('stat_cr', lambda x: (x == 'money_back').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_cr=('stat_cr', lambda x: (x != 'money_back').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] != 'money_back'].sum()),
    # Métrica de "credit lapse" (diferencia entre reimbursement_date y created_at)
    credit_lapse=('reimbursement_date', lambda x: (x - df_jo.loc[x.index, 'created_at']).dt.days.mean()),
    # Métrica de "credit payment lapse" (diferencia entre money_back_date y reimbursement_date)
    credit_payment_lapse=('money_back_date', lambda x: (x - df_jo.loc[x.index, 'reimbursement_date']).dt.days.mean())

).reset_index()

# Calcular los porcentajes de non_paid_fees
cohort_analysis['paid_fees_percent_qty'] = (cohort_analysis['paid_fees'] / (cohort_analysis['paid_fees'] + cohort_analysis['non_paid_fees']))
cohort_analysis['paid_fees_percent_amount'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['Fees'])   # % en monto
cohort_analysis['paid_cr_percent_qty'] = (cohort_analysis['paid_cr'] / (cohort_analysis['paid_cr'] + cohort_analysis['non_paid_cr']))
cohort_analysis['paid_cr_percent_amount'] = (cohort_analysis['total_paid_cr'] / cohort_analysis['CR'])   # % en monto
cohort_analysis['index'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['total_paid_cr'])   # % index

# Eliminar las columnas de pago y no pago
cohort_sizes_monthly = cohort_analysis.drop(
    ['paid_fees', 'non_paid_fees', 'total_paid_fees', 'total_non_paid_fees','paid_cr', 'non_paid_cr', 'total_paid_cr', 'total_non_paid_cr'], axis=1, errors='ignore')

# Mostrar los resultados de las cohortes por mes
#cohort_analysis

In [ ]:
cohort_analysis
#display(cohort_analysis[cohort_analysis.user_id == 2002])

In [ ]:
df = pm.df('df_jo')
#df.info()

cr_id = ['id_cr','id_fe','user_id', 'created_at','created_at_fe','amount','fee','stat_cr','stat_fe','transfer_type','type',
            'to_receive_ini', 'to_receive_bank','to_reimbur','to_reimbur_cash','to_end','to_send',
             'send_at', 'cr_received_date', 'money_back_date', 'reimbursement_date',
            'paid_at','charge_moment','moderated_at','reason','category','from_date','to_date', 'recovery_status'] # ,'id_y','from_date','to_date','reason','recovery_status', 'cash_request_id'

print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-18264]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[cr_id].sort_values('created_at'))

user_ids = [54879, 9900458, 2002, 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

pd.options.display.max_columns = None
print("Casos segun Cash User ID")
for id in (user_ids):
    #df_t = df[df['user_id'] == id].reset_index()
    df_t = df[(df['user_id'] == id) & (df['stat_cr'] == 'money_back')].reset_index()
    print(f"User ID: {id}")
    display(df_t[cr_id].sort_values('created_at').reset_index())